In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
style.use('seaborn-whitegrid')

device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
def first_rbf_parameter( input_data, target): # done
        
    find_index_input = input_data.clone().detach()
    fine_index_target = target.clone().detach()
    find_sigma = target.clone().detach()
    
    target_std = target.std()
    
    find_weight = target.clone().detach()
    center_index_list = []

    for i in range(5):
        index_ = torch.argmax(torch.sum(torch.abs(fine_index_target), dim = 0)).cpu().detach().tolist()
        fine_index_target[:,:,index_] = 0
        center_index_list.append(index_)

    center_index_list = torch.tensor(center_index_list, device=device)
    initcenter = torch.index_select(find_index_input, 0, center_index_list)
    initweight = torch.index_select(find_weight, 2, center_index_list)
                    
    sigma_list = []
    dft = torch.log(torch.abs(torch.fft.fft(find_sigma).real))
    dft =  torch.abs(dft / torch.max(dft)) **-1
    for k in center_index_list:
        sigma_list.append(torch.mean(dft[:,:,k]).reshape(1))

    initsigma = torch.cat(sigma_list)
    
    return initcenter[-5:], initweight[-5:], initsigma[-5:]

In [24]:
import numpy as np
a = np.arange(0,10,0.05)
    
y = 2*a**3 - 16*a**2 + 2*a -10
y2 = 3*a**2 -16*a - 30
y3 = 5*a**2 +25*a - a**3

target = np.array((y,y2,y3))
target = torch.tensor(target, device = device).reshape(3,1,200)
input_ = torch.tensor(a, device = device)

In [25]:
cent, weig, sig = first_rbf_parameter(input_, target)
radial_layer_centers = cent.reshape(5, 1)
radial_sigma = sig.reshape(5, 1)
weights = weig.reshape(target.size(0),5)

In [35]:
def rbf_gaussian(input_data):
    output = torch.exp(-1 * (torch.pow((input_ - radial_layer_centers), 2) / \
                                (torch.pow(radial_sigma, 2))))

    return output  # size = (num_rbf, 1)

In [43]:
rbf_output = rbf_gaussian(input_)

In [47]:
B = torch.mm(weights, rbf_output)

In [48]:
B.size()

torch.Size([3, 200])

In [69]:
target.size()

torch.Size([3, 1, 200])

In [12]:
def first_grad(input_data, target):  
    space = (input_data,)
    f_grad = torch.gradient(target, spacing = space, dim =2,edge_order  = 1)
    return f_grad[0]

In [32]:
fg = first_grad(input_, target).size()

In [29]:
def rbf_gradient(input_data, centers, sigmas, weights): # done
    rbf_output = (-2 * (input_data-centers) / torch.pow(sigmas,2)) * \
                    (torch.exp(-1 * (torch.pow((input_data - centers), 2) / (torch.pow(sigmas, 2)))))
    rbf_grad = torch.mm(weights.reshape(3, 5), rbf_output)
    
    return rbf_grad.reshape(3, 1, input_data.size(-1))

In [43]:
rg = rbf_gradient(input_, radial_layer_centers, radial_sigma, weights)

In [44]:
rg.size()

torch.Size([3, 1, 200])

In [ ]:
def rbf_gaussian(input_data):
    output = torch.exp(-1 * (torch.pow((input_ - radial_layer_centers), 2) / \
                                (torch.pow(radial_sigma, 2))))

    return output  # size = (num_rbf, 1)

def rbf_gradient(input_data, centers, sigmas, weights): # done
    rbf_output = (-2 * (input_data-centers) / torch.pow(sigmas,2)) * \
                    (torch.exp(-1 * (torch.pow((input_data - centers), 2) / (torch.pow(sigmas, 2)))))
    rbf_grad = torch.mm(weights.reshape(3, 5), rbf_output)
    
    return rbf_grad.reshape(3, 1, input_data.size(-1))

def first_grad(input_data, target): #done
    space = (input_data,)
    f_grad = torch.gradient(target[0], spacing = space, dim = 2, edge_order  = 1)
    return f_grad[0]
    
def target_grad(input_data, centers, sigmas, weights, f_grad): # done
    true_grad = f_grad - rbf_gradient(input_data, centers, sigmas, weights)
    
    return true_grad

In [45]:
fg = first_grad(input_, target)

In [46]:
fg.size()

torch.Size([3, 1, 200])

In [131]:
def forward(input_data): # done
    rbf_output = rbf_gaussian(input_data)
    pred = torch.mm(weights.reshape(3, 5), rbf_output)

    return rbf_output, pred

In [132]:
R, pred  = forward(input_)

RuntimeError: The size of tensor a (200) must match the size of tensor b (5) at non-singleton dimension 0

In [40]:
R.size()

torch.Size([5, 200])

In [41]:
pred.size()

torch.Size([3, 200])

In [48]:
L2_2_error = -2 * (fg - rg)

In [49]:
L2_2_error.size()

torch.Size([3, 1, 200])

In [55]:
pred.size()

torch.Size([3, 200])

In [56]:
target.size()

torch.Size([3, 1, 200])

In [57]:
L2_1_error = -2 * (target - pred.reshape(3,1,200))
L2_1_error.size()

torch.Size([3, 1, 200])

In [50]:
def rbf_gaussian_derivative_sigma( input_data): # done
    output = (2 * torch.pow((input_data - radial_layer_centers), 2) / \
                (torch.pow(radial_sigma, 3))) * rbf_gaussian(input_data)

    return output  # size = (num_radial, 1)

In [53]:
weights.size()

torch.Size([3, 5])

In [52]:
rbf_gaussian_derivative_sigma(input_)

torch.Size([5, 200])

In [66]:
deltaSigma1 = rbf_gaussian_derivative_sigma(input_) * L2_1_error
deltaSigma1 *= weights[0].reshape(3, 5, 1)


In [67]:
deltaSigma1.size()

torch.Size([5, 200])

In [69]:
def L2_F( input_data): #done
    return -2 * (input_data - radial_layer_centers) / torch.pow(radial_sigma, 2)

In [71]:
L2_F(input_).size()

torch.Size([5, 200])

In [72]:
deltaSigma2 = rbf_gaussian_derivative_sigma(input_) * L2_2_error
deltaSigma2 *= L2_F(input_) * weights.reshape(3, 5, 1)

In [73]:
deltaSigma2.size()

torch.Size([3, 5, 200])

In [122]:
deltaSigma1 = rbf_gaussian_derivative_sigma(input_) * L2_1_error
deltaSigma1 *= weights.reshape(3, 5, 1)

deltaSigma2 = rbf_gaussian_derivative_sigma(input_) * L2_2_error
deltaSigma2 *= L2_F(input_) * weights.reshape(3, 5, 1)
deltaSigma =  torch.mean(torch.sum(deltaSigma1, dim=2) + torch.sum(deltaSigma2, dim=2), dim = 0)

In [123]:
deltaSigma.size()

torch.Size([5])

In [86]:
torch.mean(deltaSigma, dim = 0).size()

torch.Size([5])

In [87]:
weights.size()

torch.Size([3, 5])

In [90]:
L2_1_error.size()

torch.Size([3, 1, 200])

In [91]:
(R * L2_1_error).size()

torch.Size([3, 5, 200])

In [92]:
delta_weight1 = torch.sum((R * L2_1_error)[0], dim=1)
delta_weight1.size()

torch.Size([5])

In [93]:
delta_weight1 = torch.sum((R * L2_1_error), dim=2)
delta_weight1.size()

torch.Size([3, 5])

In [88]:
delta_weight1 = torch.sum((R * L2_1_error), dim=2)
# delta_weight1 = delta_weight1.reshape(3, 1, 5)

In [94]:
def L2_2_derivateive_weight( input_data, radial_output):
    return (-2 *(input_data - radial_layer_centers) / torch.pow(radial_sigma,2)) * radial_output

In [97]:
L2_2_derivateive_weight(input_, R).size()

torch.Size([5, 200])

In [101]:
L2_2_error[0].reshape(1,200)

tensor([[-2.4100e+00, -8.4000e-01,  2.2700e+00,  5.3200e+00,  8.3100e+00,
          1.1240e+01,  1.4110e+01,  1.6920e+01,  1.9670e+01,  2.2360e+01,
          2.4990e+01,  2.7560e+01,  3.0070e+01,  3.2520e+01,  3.4910e+01,
          3.7240e+01,  3.9510e+01,  4.1720e+01,  4.3870e+01,  4.5960e+01,
          4.7990e+01,  4.9960e+01,  5.1870e+01,  5.3720e+01,  5.5510e+01,
          5.7240e+01,  5.8910e+01,  6.0520e+01,  6.2070e+01,  6.3560e+01,
          6.4990e+01,  6.6360e+01,  6.7670e+01,  6.8920e+01,  7.0110e+01,
          7.1240e+01,  7.2310e+01,  7.3320e+01,  7.4270e+01,  7.5160e+01,
          7.5990e+01,  7.6760e+01,  7.7470e+01,  7.8120e+01,  7.8710e+01,
          7.9241e+01,  7.9711e+01,  8.0121e+01,  8.0471e+01,  8.0761e+01,
          8.0992e+01,  8.1162e+01,  8.1272e+01,  8.1323e+01,  8.1313e+01,
          8.1244e+01,  8.1115e+01,  8.0926e+01,  8.0677e+01,  8.0368e+01,
          8.0000e+01,  7.9572e+01,  7.9084e+01,  7.8537e+01,  7.7930e+01,
          7.7264e+01,  7.6538e+01,  7.

In [111]:
weights.size()

torch.Size([3, 5])

In [109]:
torch.sum((L2_2_derivateive_weight(input_, R)) * L2_2_error, dim = 2).size()

torch.Size([3, 5])

In [89]:
delta_weight2 = torch.sum((L2_2_derivateive_weight(input_data, R) * L2_2_error), dim = 1)
delta_weight2 = delta_weight2.reshape(3, 1, 5)
delta_weight = delta_weight1 + delta_weight2

torch.Size([3, 200])

In [115]:
delta_weight1 = torch.sum((R * L2_1_error), dim=2)  #(in_feature, add_rbf_num)
# delta_weight1 = delta_weight1.reshape(3, 1, 5)

delta_weight2 = torch.sum((L2_2_derivateive_weight(input_, R) * L2_2_error), dim = 2)
# delta_weight2 = delta_weight2.reshape(3, 1, 5)
delta_weight = delta_weight1 + delta_weight2

In [116]:
delta_weight

tensor([[ 74542.1486,  82628.6870,  87331.9519,  90447.3154,  92661.7393],
        [ 21159.3312,  23089.9276,  24091.8163,  24626.7304,  24833.5405],
        [-45206.4535, -50172.5155, -53094.2915, -55072.8345, -56551.7174]],
       device='cuda:0', dtype=torch.float64)

In [125]:
add_rbf_number = 5

In [147]:
def first_rbf_parameter(input_data, target): # done
        
    find_index_input = input_data.clone().detach()
    fine_index_target = target.clone().detach()
    find_sigma = target.clone().detach()
    
    target_std = target.std()
    
    find_weight = target.clone().detach()
    center_index_list = []

    for i in range(add_rbf_number):
        index_ = torch.argmax(torch.sum(torch.abs(fine_index_target), dim = 0)).cpu().detach().tolist()
        fine_index_target[:,:,index_] = 0
        center_index_list.append(index_)
    print(center_index_list)

    center_index_list = torch.tensor(center_index_list, device=device)
    initcenter = torch.index_select(find_index_input, 0, center_index_list)
    initweight = torch.index_select(find_weight, 2, center_index_list)
                    
    sigma_list = []
    dft = torch.log(torch.abs(torch.fft.fft(find_sigma).real))
    dft =  torch.abs(dft / torch.max(dft)) **-1
    for k in center_index_list:
        sigma_list.append(torch.mean(dft[:,:,k]).reshape(1))

    initsigma = torch.cat(sigma_list)


    return initcenter[-add_rbf_number:].reshape(add_rbf_number,1), initweight[-add_rbf_number:].reshape(3, add_rbf_number), initsigma[-add_rbf_number:].reshape(add_rbf_number, 1)

In [148]:
radial_layer_centers, weights, radial_sigma = first_rbf_parameter(input_, target)

[199, 198, 197, 196, 195]


In [149]:
# input, target size
print(input_.size())
print(target.size())

torch.Size([200])
torch.Size([3, 1, 200])


In [150]:
# initial size
print(radial_layer_centers.size())
print(radial_sigma.size())
print(weights.size())

torch.Size([5, 1])
torch.Size([5, 1])
torch.Size([3, 5])


In [162]:
def rbf_gaussian(input_data):
    output = torch.exp(-1 * (torch.pow((input_ - radial_layer_centers), 2) / \
                                (torch.pow(radial_sigma, 2))))

    return output  # size = (num_rbf, 1)

def forward(input_data): # done
    rbf_output = rbf_gaussian(input_data)
    pred = torch.mm(weights, rbf_output)

    return rbf_output, pred

def rbf_gradient(input_data, centers, sigmas, weights): # done
    rbf_output = (-2 * (input_data-centers) / torch.pow(sigmas,2)) * \
                    (torch.exp(-1 * (torch.pow((input_data - centers), 2) / (torch.pow(sigmas, 2)))))
    rbf_grad = torch.mm(weights, rbf_output)
    
    return rbf_grad.reshape(3,1,200)

def first_grad(input_data, target): #done
    space = (input_data,)
    f_grad = torch.gradient(target, spacing = space, dim = 2, edge_order  = 1)
    return f_grad[0]
    
def target_grad(input_data, centers, sigmas, weights, f_grad): # done
    true_grad = f_grad - rbf_gradient(input_data, centers, sigmas, weights)
    
    return true_grad

In [152]:
R, pred = forward(input_)

In [153]:
print(R.size())
print(pred.size())

torch.Size([5, 200])
torch.Size([3, 200])


In [158]:
rg = rbf_gradient(input_, radial_layer_centers, radial_sigma, weights)

In [159]:
print(rg.size())

torch.Size([3, 200])


In [163]:
fg = first_grad(input_, target)

In [165]:
print(fg.size())

torch.Size([3, 1, 200])


In [166]:
L2_1_error = -2 * (target - pred.reshape(3,1,200))
L2_2_error = -2 * (fg - rg.reshape(3,1,200))

In [168]:
print(L2_1_error.size())
print(L2_2_error.size())

torch.Size([3, 1, 200])
torch.Size([3, 1, 200])


In [213]:
def L2_F( input_data):
    return -2 * (input_data - radial_layer_centers) / torch.pow(radial_sigma, 2)

In [215]:
L2_F(input_).size()

torch.Size([5, 200])

In [169]:
def rbf_gaussian_derivative_sigma( input_data): # done
    output = (2 * torch.pow((input_data - radial_layer_centers), 2) / \
                (torch.pow(radial_sigma, 3))) * rbf_gaussian(input_data)

    return output  # size = (num_radial, 1)

In [170]:
der_sigma = rbf_gaussian_derivative_sigma(input_)
print(der_sigma.size())

torch.Size([5, 200])


In [184]:
(rbf_gaussian_derivative_sigma(input_) * L2_1_error)[0] * weights.reshape(3, add_rbf_number, 1)[0]

tensor([[1.0165e-31, 2.3548e-31, 5.4744e-31, 1.2769e-30, 2.9871e-30, 7.0050e-30,
         1.6458e-29, 3.8710e-29, 9.1087e-29, 2.1427e-28, 5.0353e-28, 1.1813e-27,
         2.7651e-27, 6.4541e-27, 1.5016e-26, 3.4805e-26, 8.0351e-26, 1.8469e-25,
         4.2259e-25, 9.6232e-25, 2.1806e-24, 4.9160e-24, 1.1026e-23, 2.4599e-23,
         5.4591e-23, 1.2050e-22, 2.6455e-22, 5.7768e-22, 1.2546e-21, 2.7099e-21,
         5.8217e-21, 1.2439e-20, 2.6434e-20, 5.5873e-20, 1.1746e-19, 2.4562e-19,
         5.1086e-19, 1.0569e-18, 2.1748e-18, 4.4518e-18, 9.0646e-18, 1.8360e-17,
         3.6994e-17, 7.4151e-17, 1.4786e-16, 2.9331e-16, 5.7883e-16, 1.1364e-15,
         2.2197e-15, 4.3136e-15, 8.3399e-15, 1.6043e-14, 3.0703e-14, 5.8464e-14,
         1.1077e-13, 2.0880e-13, 3.9163e-13, 7.3088e-13, 1.3572e-12, 2.5077e-12,
         4.6103e-12, 8.4339e-12, 1.5352e-11, 2.7807e-11, 5.0119e-11, 8.9885e-11,
         1.6041e-10, 2.8486e-10, 5.0339e-10, 8.8518e-10, 1.5489e-09, 2.6970e-09,
         4.6731e-09, 8.0575e

In [190]:
deltaSigma1[2] - (rbf_gaussian_derivative_sigma(input_) * L2_1_error)[2] * weights.reshape(3, add_rbf_number, 1)[2]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 

In [179]:
deltaSigma1 = rbf_gaussian_derivative_sigma(input_) * L2_1_error
print(deltaSigma1.size())
deltaSigma1 *= weights.reshape(3, add_rbf_number, 1)
print(deltaSigma1.size())

deltaSigma2 = rbf_gaussian_derivative_sigma(input_) * L2_2_error
print(deltaSigma2.size())
deltaSigma2 *= L2_F(input_) * weights.reshape(3, add_rbf_number, 1)
print(deltaSigma2.size())
deltaSigma =  torch.mean(torch.sum(deltaSigma1, dim=2) + torch.sum(deltaSigma2, dim=2), dim = 0) # (add_rbf_num)

torch.Size([3, 5, 200])
torch.Size([3, 5, 200])
torch.Size([3, 5, 200])
torch.Size([3, 5, 200])


In [180]:
print(deltaSigma.size())

torch.Size([5])


In [191]:
def L2_2_derivateive_weight( input_data, radial_output):
    return (-2 *(input_data - radial_layer_centers) / torch.pow(radial_sigma,2)) * radial_output

In [192]:
L2_2_derivateive_weight(input_, R).size()

torch.Size([5, 200])

In [198]:
torch.sum((R * L2_1_error), dim = 2)

tensor([[ 49743.4251,  57699.7949,  63618.1871,  68744.7548,  73826.4180],
        [ 13511.3828,  15526.3073,  16973.1939,  18175.6782,  19303.8351],
        [-30360.4084, -35265.8156, -38940.2653, -42153.1133, -45384.9690]],
       device='cuda:0', dtype=torch.float64)

In [200]:
torch.sum((R * L2_1_error)[2], dim = 1)

tensor([-30360.4084, -35265.8156, -38940.2653, -42153.1133, -45384.9690],
       device='cuda:0', dtype=torch.float64)

In [204]:
L2_2_error.size()

torch.Size([3, 1, 200])

In [201]:
delta_weight2 = L2_2_derivateive_weight(input_, R) * L2_2_error

In [209]:
delta_weight2[2] - L2_2_derivateive_weight(input_, R) * L2_2_error[2] 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 

In [202]:
delta_weight2.size()

torch.Size([3, 5, 200])

In [ ]:
delta_weight1 = torch.sum((radial_output * L2_1_error), dim=2)        # (in_feature, add_rbf_num)
delta_weight2 = torch.sum((L2_2_derivateive_weight(input_, R) * L2_2_error), dim = 2) # (in_feature, add_rbf_num)
delta_weight = delta_weight1 + delta_weight2 # (in_feature, add_rbf_num)

In [226]:
def Loss(pred, target, pred_grad, true_grad): # ?
    # value loss + gradient loss 

    return torch.mean(torch.pow(target - pred.reshape(3,1,200),2) + torch.pow(true_grad - pred_grad.reshape(3,1,200), 2))

In [227]:
Loss(pred, target, rg, fg)

tensor(163445.3860, device='cuda:0', dtype=torch.float64)

In [228]:
Loss(pred, target, rg, fg)

tensor(163445.3860, device='cuda:0', dtype=torch.float64)

In [233]:
weights

tensor([[ 321.4676,  299.6093,  281.3513,  264.8967,  249.5570],
        [  86.6482,   82.5401,   79.3757,   76.6933,   74.3540],
        [-196.1059, -182.5765, -171.2148, -160.9192, -151.2452]],
       device='cuda:0', dtype=torch.float64)

In [230]:
list_weight = [weights, weights]

In [232]:
torch.cat(list_weight, dim = 1)

tensor([[ 321.4676,  299.6093,  281.3513,  264.8967,  249.5570,  321.4676,
          299.6093,  281.3513,  264.8967,  249.5570],
        [  86.6482,   82.5401,   79.3757,   76.6933,   74.3540,   86.6482,
           82.5401,   79.3757,   76.6933,   74.3540],
        [-196.1059, -182.5765, -171.2148, -160.9192, -151.2452, -196.1059,
         -182.5765, -171.2148, -160.9192, -151.2452]], device='cuda:0',
       dtype=torch.float64)